# Introduction

In [ ]:
#Import your tensorflow as tf 
...
import matplotlib.pyplot as plt
import datetime
import random
import numpy as np
%matplotlib inline

In [ ]:
#Downloading MNIST Dataset into "MNIST_data" and one_hot =TRUE Indicates output is of form [0 0 0 0 0 1 0 0 0 0] for 5

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Inputs and Outputs

Your x and y_ are just going to place placeholders that basically just indicate the type of input you want in your CNN and the type of output. For each of these placeholders, you have to specify the type and the shape. 

In [ ]:
tf.reset_default_graph() 
sess = tf.InteractiveSession()
# Example
'''
tf.placeholder(
    dtype,
    shape=None,
    name=None
)

'''

# Create placeholder for x (input) of dtype float and shape [None,28,28,1]
 ...                                                   #shape in CNNs is always None x height x width x color channels

    
#Create placeholder for y (output) of dtype float and shape [None, 10]
 ...                                                    #shape is always None x number of classes

# Network Architecture

Now that we have our placeholders, we just have to specify the network architecture. Basically, the main point we have to remember is that all of the filters (weights) and biases are tensorflow variables. Let's create our filter and bias for the first layer

In [ ]:
# Create a Variable W_conv1 which is randomly initialised via truncated normal
# shape is filter x filter x input channels x output channels


#W_conv1 = tf.Variable(tf.truncated_normal([filter, filter, input, output], stddev=0.1))
...

# Create a Variable b_conv1
b_conv1 = tf.Variable(tf.constant(.1, shape = [32])) #shape of the bias just has to match output channels of the filter

Now that we have our filter and our bias, we can call our first conv layer. The 4 arguments you have to specify are the input (which is where our placeholder comes into play), the filter (we just created the variable for that), the stride, and the padding)

### tf.nn.conv2d(input, filter, strides, padding, name=None )
#### Input: A Tensor. Must be one of the following types: half, bfloat16, float32. A 4-D tensor. 
#### filter: A Tensor. Must have the same type as input. A 4-D tensor of shape [filter_height, filter_width, in_channels, out_channels]
#### strides: A list of ints. 1-D tensor of length 4. The stride of the sliding window for each dimension of input. The dimension order is determined by the value of data_format, see below for details.
#### padding: A string from: "SAME", "VALID". The type of padding algorithm to use.



In [ ]:
print (x)
print (W_conv1)

#Create conv2d layer with input = x, filter = W_conv1, strides = [1,1,1,1] padding = "SAME"

h_conv1 = ... + b_conv1

#Apply relu activation function
h_conv1 = tf.nn.relu(...)

h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

This just defines some methods to make the function calls a little nicer. 

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(input=x, filter=W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Now, let's just complete the network

In [ ]:
#Second Conv and Pool Layers
#W_conv2 = tf.Variable(tf.truncated_normal([filter, filter, input, output], stddev=0.1)) 
W_conv2 = ...

b_conv2 = ...

h_conv2 = tf.nn.relu(conv2d(... , ... ) + b_conv2)

h_pool2 = max_pool_2x2(...)

#First Fully Connected Layer
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(.1, shape = [1024]))
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#Dropout Layer
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#Second Fully Connected Layer
# Create tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))

W_fc2 = ...

b_fc2 = tf.Variable(tf.constant(.1, shape = [10]))

#Final Layer
y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

Now, let's formulate our loss function

In [ ]:
'''
x = tf.constant([[1., 1.], [2., 2.]])
tf.reduce_mean(x)  # 1.5
tf.reduce_mean(x, 0)  # [1.5, 1.5]
tf.reduce_mean(x, 1)  # [1.,  2.]

Cross entropy
H(y)=∑ y' * log(1/y)= − ∑y' * logy

'''
# tf.reduce_mean()
#tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y)

crossEntropyLoss = ... 

Now, we need to use an optimizer to minimize the function

In [ ]:
# Train using tf.train and optimizer used is AdamOptimizer to minimize crossEntropyLoss

trainStep = ...

The following are the statements that help with calculating accuracy

In [ ]:
# tf.argmax(input, axis=None, name=None, dimension=None)
# Returns the index with the largest value across axis of a tensor

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

The next line is the main statement that gets initializes all the variables we've declared earlier

In [ ]:
sess.run(tf.global_variables_initializer())

This is code to allow us to visualize our training with Tensorboard

In [ ]:
tf.summary.scalar('Cross Enropy Loss', crossEntropyLoss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [ ]:
b = mnist.train.next_batch(1)
print (b[0].shape) #b[0] contains the image
image = tf.reshape(b[0], [-1,28,28,1])
print (image)
my_img = image.eval() #here is your image Tensor
my_i = my_img.squeeze()
plt.imshow(my_i, cmap='gray_r')
plt.show()

# Training

In [ ]:
batchSize = 50
for i in range(1000):
    batch = mnist.train.next_batch(batchSize)
    trainingInputs = batch[0].reshape([batchSize,28,28,1])
    trainingLabels = batch[1]
    if i%10 == 0:
        summary = sess.run(merged, {x: trainingInputs, y_: trainingLabels, keep_prob: 1.0})
        writer.add_summary(summary, i)
    if i%100 == 0:
        trainAccuracy = accuracy.eval(session=sess, feed_dict={x:trainingInputs, y_: trainingLabels, keep_prob: 1.0})
        print ("step %d, training accuracy %g"%(i, trainAccuracy))
    trainStep.run(session=sess, feed_dict={x: trainingInputs, y_: trainingLabels, keep_prob: 0.5})

# Testing

In [ ]:
testInputs = mnist.test.images.reshape([-1, 28, 28, 1])
testLabels = mnist.test.labels
acc = accuracy.eval(feed_dict = {x: testInputs, y_: testLabels, keep_prob: 1.0})
print("testing accuracy: {}".format(acc))